In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob, skimage, dataset, cv2
from skimage import io, transform
from tqdm import tqdm
from copy import copy
from tools import *
from reload import *
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

%matplotlib inline

In [2]:
dataset = reload()

[ Load Dataset ]: 100%|██████████| 5067/5067 [00:16<00:00, 308.55it/s]


In [3]:
projected_dataset = project(dataset.train_dataset, 'y')

In [4]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 64*64])
Y = tf.placeholder(tf.int32, [None, 1])
Y_one_hot=tf.one_hot(Y,depth=5)
Y_ont_hot=tf.reshape(Y_one_hot,[-1,5])
keep_prob = tf.placeholder(tf.float32)

In [5]:
tf.set_random_seed(777)

W1 = tf.get_variable("1", shape=[4096, 2048],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([2048]))

W2 = tf.get_variable("2", shape=[2048, 1024],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([1024]))

W3 = tf.get_variable("3", shape=[1024, 512],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))

W4 = tf.get_variable("4", shape=[512, 256],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([256]))

W5 = tf.get_variable("5", shape=[256, 128],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([128]))

W6 = tf.get_variable("6", shape=[128, 64],initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([64]))

W7 = tf.get_variable("7", shape=[64, 5],initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([5]))

In [6]:
# Layer 구성
L1=tf.matmul(X, W1) + b1
bn1 = tf.contrib.layers.batch_norm(L1)
L1=tf.nn.relu(bn1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

L2=tf.matmul(L1, W2) + b2
bn2 = tf.contrib.layers.batch_norm(L2)
L2=tf.nn.relu(bn2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

L3=tf.matmul(L2, W3) + b3
bn3 = tf.contrib.layers.batch_norm(L3)
L3 = tf.nn.relu(bn3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

L4=tf.matmul(L3, W4) + b4
bn4 = tf.contrib.layers.batch_norm(L4)
L4 = tf.nn.relu(bn4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

L5=tf.matmul(L4, W5) + b5
bn5 = tf.contrib.layers.batch_norm(L5)
L5 = tf.nn.relu(bn5)
L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

L6=tf.matmul(L5, W6) + b6
bn6 = tf.contrib.layers.batch_norm(L6)
L6 = tf.nn.relu(bn6)
L6 = tf.nn.dropout(L6, keep_prob=keep_prob)

logit = tf.matmul(L6, W7) + b7
hypothesis=tf.nn.softmax(logit)

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y_one_hot))
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

prediction=tf.argmax(hypothesis,1)
target=tf.argmax(Y,1)

correct =tf.equal(prediction, target)
accuracy=tf.reduce_mean(tf.cast(correct, tf.float32))

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
training_epochs = 100000

indices = np.random.permutation(len(projected_dataset))
projected_dataset = projected_dataset[indices]
dataset.train_labels = dataset.train_labels[indices]

In [ ]:
for epoch in range(10000000):
    with tf.device('/device:GPU:2'):
        feed_dict = {X: projected_dataset.reshape(len(projected_dataset),-1), Y: dataset.train_labels, keep_prob:0.7}
        c, _, tr_acc = sess.run([cost, optimizer, accuracy], feed_dict=feed_dict)
        if epoch != 0 and epoch%1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'acc = ','{:.5f}'.format(tr_acc), 'cost =', '{:.5f}'.format(c))

print('Learning Finished!')

Epoch: 1001 acc =  0.00148 cost = 1.24163
Epoch: 2001 acc =  0.00148 cost = 1.14425
Epoch: 3001 acc =  0.00030 cost = 1.11026
Epoch: 4001 acc =  0.00089 cost = 1.07753
Epoch: 5001 acc =  0.00000 cost = 1.04400
Epoch: 6001 acc =  0.00089 cost = 1.03470
Epoch: 7001 acc =  0.00030 cost = 0.99830
Epoch: 8001 acc =  0.00030 cost = 0.96099
Epoch: 9001 acc =  0.00030 cost = 0.95013
Epoch: 11001 acc =  0.00030 cost = 0.88146
Epoch: 12001 acc =  0.00000 cost = 0.86777
Epoch: 13001 acc =  0.00119 cost = 0.83247
Epoch: 14001 acc =  0.00326 cost = 0.81108
Epoch: 15001 acc =  0.00504 cost = 0.79391
Epoch: 16001 acc =  0.00978 cost = 0.79061
Epoch: 17001 acc =  0.00889 cost = 0.77649
Epoch: 18001 acc =  0.01244 cost = 0.77045
Epoch: 19001 acc =  0.01748 cost = 0.76486
Epoch: 20001 acc =  0.02015 cost = 0.75510
Epoch: 21001 acc =  0.02193 cost = 0.73803
Epoch: 22001 acc =  0.02252 cost = 0.72981
Epoch: 23001 acc =  0.01896 cost = 0.70708
Epoch: 25001 acc =  0.01956 cost = 0.68141
Epoch: 26001 acc =  